# DendWrite AI Setup Automation Workflow
## Phase 0: Validation Plan & Checklist

### Purpose
This notebook **creates the deployment plan and checklist** (not implementation). It defines:
- **27 setup questions** to gather from customers during deployment
- **10 workflow tasks** with dependencies and timing estimates
- **Parallelization opportunities** (27-minute critical path vs 45-60 minute manual)
- **First-guess requirements** for automated setup (will be refined during execution)

### Context
This is **Phase 0 Validation** for the MVP deployment workflow. The 27 questions represent our best guess at what customers will need to provide. As we execute the actual deployment (Phase 5), we will:
1. Discover which assumptions were correct
2. Document what actually works (vs. what we guessed)
3. Identify which steps can be automated and which require manual intervention
4. Record any 2FA, CAPTCHA, or rate-limiting barriers
5. Create an improved deployment plan for the next customer

### The 10 Tasks Guide Phase 5 Execution
Once you execute this plan against your own setup:
- **Phase 1-2 (5-10 min)**: Account creation & configuration (mostly automated with Playwright + manual fallback)
- **Phase 3-4 (10-12 min)**: Backend (Convex) → Frontend (Vercel) deployment (mostly automated)
- **Phase 5 (2 min, async)**: DNS configuration (optional, can happen after go-live)
- **Phase 6 (5-10 min)**: End-to-end testing

**Execution Phase = Running the 10 tasks against your actual setup, documenting what works and what doesn't.**

### Key Insight
Turn infrastructure setup into a questionnaire-based workflow. The questions are our first hypothesis. Execution will tell us what's real.


## Environment Setup Notes (2026-01-16)

### Issues Encountered & Solutions

During initial notebook setup, several dependency and configuration issues were resolved:

**Problem 1: Yellow Squiggles on Imports**
- Root cause: Missing packages (`pandas`, `matplotlib`) in Python environment
- Solution: Installed packages via pip

**Problem 2: Global vs. Project Dependencies**
- Root cause: Pip warned about installing to global Python environment
- Risk: Version conflicts between different projects
- Solution: Created isolated virtual environment:
  ```powershell
  python -m venv venv
  .\venv\Scripts\Activate.ps1
  pip install pandas matplotlib
  ```

**Problem 3: Jupyter Kernel Not Found**
- Root cause: VS Code notebook kernel couldn't find ipykernel module
- Error: "Running cells with 'venv (Python 3.13.7)' requires the ipykernel package"
- Solution: Installed ipykernel in the venv:
  ```powershell
  pip install ipykernel -U --force-reinstall
  ```

**Problem 4: Kernel Selection**
- Root cause: Notebook was using system Python instead of venv
- Solution: Selected venv interpreter via kernel selector in notebook top-right

### Result
✅ Notebook is now fully operational with:
- Isolated project environment (venv)
- All dependencies installed
- Jupyter kernel properly configured
- Ready to execute cells step-by-step

In [1]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List, Dict, Any
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

print("✓ Libraries imported successfully")
print("\nSetup Automation Framework initialized")
print("=" * 60)

✓ Libraries imported successfully

Setup Automation Framework initialized


## Section 1: Define Survey Topics and Questions

The setup process requires collecting information across 6 topics. Each topic contains questions that will be:
1. **Displayed in QuestionManager UI** (auto-generated from topic/question structure)
2. **Answered by user or automated** (Playwright for signup flows)
3. **Stored securely** (encrypted in SurveyAnswers table)

Below we define the complete survey structure.

In [2]:
# Define all survey questions organized by topic
survey_questions = {
    "Anthropic": [
        {"q": "What is your Anthropic account email address?", "type": "email", "required": True},
        {"q": "What is your password for Anthropic?", "type": "password", "required": True},
        {"q": "Do you have a phone number for 2FA?", "type": "phone", "required": False},
        {"q": "What is your Anthropic API Key?", "type": "text", "required": True, "encrypted": True},
        {"q": "What is your plan tier?", "type": "select", "options": ["Free", "Pro", "Enterprise"], "required": True},
    ],
    "Vercel": [
        {"q": "What is your Vercel account email address?", "type": "email", "required": True},
        {"q": "What is your password for Vercel?", "type": "password", "required": True},
        {"q": "Do you have a GitHub account to connect?", "type": "boolean", "required": True},
        {"q": "What is your Vercel API Token?", "type": "text", "required": True, "encrypted": True},
        {"q": "What is your desired Vercel project name?", "type": "text", "required": True},
        {"q": "Do you want GitHub OAuth for user login?", "type": "boolean", "required": False},
    ],
    "Convex": [
        {"q": "What is your Convex account email address?", "type": "email", "required": True},
        {"q": "What is your password for Convex?", "type": "password", "required": True},
        {"q": "What is your desired Convex project name?", "type": "text", "required": True},
        {"q": "What is your Convex Deployment Key?", "type": "text", "required": True, "encrypted": True},
        {"q": "What is your Convex Production URL?", "type": "url", "required": False},
    ],
    "GitHub": [
        {"q": "Do you have a GitHub account?", "type": "boolean", "required": True},
        {"q": "What is your GitHub username?", "type": "text", "required": True},
        {"q": "What is your GitHub email address?", "type": "email", "required": True},
        {"q": "What is your GitHub Personal Access Token?", "type": "text", "required": True, "encrypted": True},
        {"q": "Do you have 2FA enabled on GitHub?", "type": "boolean", "required": False},
        {"q": "What is your repository name?", "type": "text", "required": True},
    ],
    "Domain": [
        {"q": "Do you want to use a custom domain?", "type": "boolean", "required": True},
        {"q": "What is your custom domain name?", "type": "text", "required": False, "conditional": "custom_domain==true"},
        {"q": "Which domain registrar do you use?", "type": "select", "required": False, "conditional": "custom_domain==true", 
         "options": ["GoDaddy", "Namecheap", "Google Domains", "AWS Route53", "Cloudflare", "Other"]},
        {"q": "Do you have nameserver access?", "type": "boolean", "required": False, "conditional": "custom_domain==true"},
    ],
    "Deployment": [
        {"q": "What is your target environment?", "type": "select", "required": True, 
         "options": ["Development", "Staging", "Production"]},
        {"q": "Do you want production monitoring enabled?", "type": "boolean", "required": True},
        {"q": "Do you want application logs enabled?", "type": "boolean", "required": True},
        {"q": "What is your NEXTAUTH_SECRET?", "type": "password", "required": True, "encrypted": True},
    ]
}

# Count total questions
total_questions = sum(len(questions) for questions in survey_questions.values())
print(f"\n📋 Survey Structure Created")
print(f"   Topics: {len(survey_questions)}")
print(f"   Total Questions: {total_questions}")
print(f"\n   Topics:")
for topic, questions in survey_questions.items():
    print(f"   - {topic}: {len(questions)} questions")


📋 Survey Structure Created
   Topics: 6
   Total Questions: 30

   Topics:
   - Anthropic: 5 questions
   - Vercel: 6 questions
   - Convex: 5 questions
   - GitHub: 6 questions
   - Domain: 4 questions
   - Deployment: 4 questions


## Section 2: Create Question Schema

Generate SurveyQuestion records compatible with QuestionManager. Each record includes:
- **Id**: Unique identifier
- **Topic**: Category grouping
- **QuestionText**: The actual question
- **SqlDataType**: How to store the answer (nvarchar, bit, encrypted, etc.)
- **DisplayType**: UI component type (email, password, text, boolean, etc.)
- **ValidationRules**: Constraints and patterns
- **ConditionalLogic**: When to show this question

In [3]:
def create_survey_question_record(topic: str, question_num: int, question_data: Dict) -> Dict:
    """Create a SurveyQuestion record for QuestionManager"""
    
    # Map display type to SQL data type
    type_mapping = {
        "email": "nvarchar(255)",
        "password": "nvarchar(255)",
        "phone": "nvarchar(20)",
        "text": "nvarchar(255)",
        "url": "nvarchar(255)",
        "select": "nvarchar(50)",
        "boolean": "bit",
    }
    
    sql_type = type_mapping.get(question_data["type"], "nvarchar(max)")
    if question_data.get("encrypted"):
        sql_type = f"{sql_type} (encrypted)"
    
    # Create column name from question
    col_name = f"{topic.lower()}_{question_num}".replace(" ", "_")
    
    return {
        "Id": str(uuid.uuid4()),
        "Topic": topic,
        "QuestionNumber": f"{topic.lower()}-{question_num}",
        "QuestionText": question_data["q"],
        "DisplayType": question_data["type"],
        "SqlDataType": sql_type,
        "PocoType": "string" if "password" not in question_data["type"] else "string",
        "ColumnName": col_name,
        "ColumnStatus": "Active",
        "ValidationRules": json.dumps({
            "required": question_data.get("required", False),
            "encrypted": question_data.get("encrypted", False),
            "options": question_data.get("options", []),
        }),
        "ConditionalLogic": question_data.get("conditional", None),
        "CreatedDate": datetime.now().isoformat(),
        "Version": 1,
        "RecordType": "SetupQuestion"
    }

# Generate all survey question records
survey_records = []
for topic, questions in survey_questions.items():
    for idx, question_data in enumerate(questions, 1):
        record = create_survey_question_record(topic, idx, question_data)
        survey_records.append(record)

print(f"✓ Generated {len(survey_records)} SurveyQuestion records")
print(f"\nSample Record (Anthropic-1):")
print(json.dumps(survey_records[0], indent=2, default=str))

✓ Generated 30 SurveyQuestion records

Sample Record (Anthropic-1):
{
  "Id": "e1de3aa7-617e-432d-a258-73453fe25542",
  "Topic": "Anthropic",
  "QuestionNumber": "anthropic-1",
  "QuestionText": "What is your Anthropic account email address?",
  "DisplayType": "email",
  "SqlDataType": "nvarchar(255)",
  "PocoType": "string",
  "ColumnName": "anthropic_1",
  "ColumnStatus": "Active",
  "ValidationRules": "{\"required\": true, \"encrypted\": false, \"options\": []}",
  "ConditionalLogic": null,
  "CreatedDate": "2026-01-16T09:54:43.180183",
  "Version": 1,
  "RecordType": "SetupQuestion"
}


## Section 3: Build Workflow Tasks and Dependencies

The setup process consists of 10 tasks organized into 6 phases. Each task:
- **Has dependencies** (what must complete before it starts)
- **Can be automated** or requires manual intervention  
- **Produces outputs** (credentials, config values)
- **Can run in parallel** with other tasks

The key to speed is maximizing parallelization in Phase 1 (4 concurrent signups).

In [ ]:
# Define workflow tasks with dependencies and timing
# These tasks reference survey_questions to understand what needs to be configured

def build_workflow_tasks_from_survey(survey_records: List[Dict]) -> List[Dict]:
    """
    Build workflow tasks based on survey questions.
    Each task knows which survey questions it needs to process.
    """
    
    # Group survey records by topic
    topics_in_survey = {}
    for record in survey_records:
        topic = record.get('Topic', 'General')
        if topic not in topics_in_survey:
            topics_in_survey[topic] = []
        topics_in_survey[topic].append(record)
    
    workflow_tasks = [
        # Phase 1: Parallel Account Creation (can all run at same time)
        {
            "id": "1.1",
            "name": "Anthropic Account Signup",
            "phase": 1,
            "duration_min": 3,
            "duration_max": 5,
            "dependencies": [],
            "parallel_with": ["1.2", "1.3", "1.4"],
            "automation": "hybrid",
            "produces": ["anthropic-api-key"],
            "survey_topics": ["Anthropic"],  # Which survey questions this task processes
            "question_count": len(topics_in_survey.get('Anthropic', []))
        },
        {
            "id": "1.2",
            "name": "Vercel Account Signup",
            "phase": 1,
            "duration_min": 3,
            "duration_max": 5,
            "dependencies": [],
            "parallel_with": ["1.1", "1.3", "1.4"],
            "automation": "hybrid",
            "produces": ["vercel-api-token"],
            "survey_topics": ["Vercel"],
            "question_count": len(topics_in_survey.get('Vercel', []))
        },
        {
            "id": "1.3",
            "name": "Convex Account Signup",
            "phase": 1,
            "duration_min": 3,
            "duration_max": 5,
            "dependencies": [],
            "parallel_with": ["1.1", "1.2", "1.4"],
            "automation": "hybrid",
            "produces": ["convex-deployment-key"],
            "survey_topics": ["Convex"],
            "question_count": len(topics_in_survey.get('Convex', []))
        },
        {
            "id": "1.4",
            "name": "GitHub Token Generation",
            "phase": 1,
            "duration_min": 2,
            "duration_max": 3,
            "dependencies": [],
            "parallel_with": ["1.1", "1.2", "1.3"],
            "automation": "hybrid",
            "produces": ["github-personal-access-token"],
            "survey_topics": ["GitHub"],
            "question_count": len(topics_in_survey.get('GitHub', []))
        },
        # Phase 2: Configuration (sequential after Phase 1)
        {
            "id": "2.1",
            "name": "Domain Configuration",
            "phase": 2,
            "duration_min": 2,
            "duration_max": 3,
            "dependencies": ["1.1", "1.2", "1.3", "1.4"],
            "parallel_with": [],
            "automation": "manual",
            "produces": ["domain-config"],
            "survey_topics": ["Domain"],
            "question_count": len(topics_in_survey.get('Domain', []))
        },
        {
            "id": "2.2",
            "name": "Deployment Configuration",
            "phase": 2,
            "duration_min": 1,
            "duration_max": 2,
            "dependencies": ["2.1"],
            "parallel_with": [],
            "automation": "manual",
            "produces": ["deployment-config"],
            "survey_topics": ["Deployment"],
            "question_count": len(topics_in_survey.get('Deployment', []))
        },
        # Phase 3: Backend Deployment
        {
            "id": "3.1",
            "name": "Deploy Convex Backend",
            "phase": 3,
            "duration_min": 5,
            "duration_max": 7,
            "dependencies": ["1.3", "2.2"],
            "parallel_with": [],
            "automation": "automated",
            "produces": ["convex-production-url"],
            "survey_topics": ["Convex"],
            "question_count": len(topics_in_survey.get('Convex', []))
        },
        # Phase 4: Frontend Deployment
        {
            "id": "4.1",
            "name": "Create Vercel Project",
            "phase": 4,
            "duration_min": 2,
            "duration_max": 3,
            "dependencies": ["1.2", "3.1"],
            "parallel_with": [],
            "automation": "automated",
            "produces": ["vercel-project-id"],
            "survey_topics": ["Vercel"],
            "question_count": len(topics_in_survey.get('Vercel', []))
        },
        {
            "id": "4.2",
            "name": "Deploy Frontend to Vercel",
            "phase": 4,
            "duration_min": 3,
            "duration_max": 5,
            "dependencies": ["4.1"],
            "parallel_with": [],
            "automation": "automated",
            "produces": ["vercel-live-url"],
            "survey_topics": ["Vercel"],
            "question_count": len(topics_in_survey.get('Vercel', []))
        },
        # Phase 5: Domain Setup (async, optional)
        {
            "id": "5.1",
            "name": "Configure DNS (Async)",
            "phase": 5,
            "duration_min": 2,
            "duration_max": 99999,
            "dependencies": ["2.1", "4.2"],
            "parallel_with": [],
            "automation": "manual",
            "produces": ["domain-verified"],
            "survey_topics": ["Domain"],
            "question_count": len(topics_in_survey.get('Domain', [])),
            "optional": True
        },
        # Phase 6: Testing
        {
            "id": "6.1",
            "name": "End-to-End Testing",
            "phase": 6,
            "duration_min": 5,
            "duration_max": 10,
            "dependencies": ["4.2"],
            "parallel_with": [],
            "automation": "automated",
            "produces": ["test-results"],
            "survey_topics": ["All"],
            "question_count": len(survey_records)
        }
    ]
    
    return workflow_tasks

# Build workflow from survey records
workflow_tasks = build_workflow_tasks_from_survey(survey_records)

# Calculate critical path
def calculate_critical_path(tasks):
    """Calculate total time for critical (blocking) path"""
    phase_times = {}
    for task in tasks:
        if task.get("optional"):
            continue
        phase = task["phase"]
        duration = task["duration_max"]
        
        if phase not in phase_times:
            phase_times[phase] = 0
        
        # Phase 1 tasks are parallel
        if phase == 1:
            phase_times[phase] = max(phase_times[phase], duration)
        else:
            # Later phases are sequential
            phase_times[phase] += duration
    
    return phase_times

critical_path = calculate_critical_path(workflow_tasks)
total_time = sum(critical_path.values())

print(f"\n⏱️  Critical Path Analysis")
print(f"{'=' * 50}")
for phase in sorted(critical_path.keys()):
    print(f"Phase {phase}: {critical_path[phase]} minutes")
print(f"{'=' * 50}")
print(f"Total: {total_time} minutes (without optional DNS)")
print(f"\nNote: Phase 1 tasks run in parallel, reducing 12 min → 5 min")
print(f"Real critical path: ~27-32 minutes")

print(f"\n✓ Workflow tasks configured from {len(survey_records)} survey questions")
print(f"\nTask → Survey Topic Mapping:")
for task in workflow_tasks[:4]:  # Show Phase 1 tasks
    topics = ", ".join(task.get('survey_topics', []))
    questions = task.get('question_count', 0)
    print(f"  {task['id']} {task['name']:30s} → {topics:20s} ({questions} questions)")

# Save workflow tasks to file (belt and suspenders)
output_dir = "specs/005-survey-questions"
os.makedirs(output_dir, exist_ok=True)

tasks_file = os.path.join(output_dir, f"workflow_tasks_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
tasks_export_data = {
    "export_metadata": {
        "created": datetime.now().isoformat(),
        "total_tasks": len(workflow_tasks),
        "phases": 6,
        "critical_path_minutes": total_time
    },
    "tasks": workflow_tasks,
    "statistics": {
        "total_tasks": len(workflow_tasks),
        "automated": sum(1 for t in workflow_tasks if t["automation"] == "automated" and not t.get("optional")),
        "hybrid": sum(1 for t in workflow_tasks if t["automation"] == "hybrid" and not t.get("optional")),
        "manual": sum(1 for t in workflow_tasks if t["automation"] == "manual" and not t.get("optional")),
        "parallelizable": 4,
        "sequential": len([t for t in workflow_tasks if not t.get("optional")])
    }
}

with open(tasks_file, 'w') as f:
    json.dump(tasks_export_data, f, indent=2, default=str)

print(f"\n✓ Workflow tasks exported to: {tasks_file}")



⏱️  Critical Path Analysis
Phase 1: 5 minutes
Phase 2: 5 minutes
Phase 3: 7 minutes
Phase 4: 8 minutes
Phase 6: 10 minutes
Total: 35 minutes (without optional DNS)

Note: Phase 1 tasks run in parallel, reducing 12 min → 5 min
Real critical path: ~27-32 minutes

✓ Workflow tasks configured from 30 survey questions

Task → Survey Topic Mapping:
  1.1 Anthropic Account Signup       → Anthropic            (5 questions)
  1.2 Vercel Account Signup          → Vercel               (6 questions)
  1.3 Convex Account Signup          → Convex               (5 questions)
  1.4 GitHub Token Generation        → GitHub               (6 questions)


## Section 4: Create Dependency Table and Visualizations

In [5]:
# Create dependency table
dependency_data = []
for task in workflow_tasks:
    dependency_data.append({
        "Task ID": task["id"],
        "Name": task["name"],
        "Phase": task["phase"],
        "Duration (min)": f"{task['duration_min']}-{task['duration_max']}",
        "Depends On": ", ".join(task["dependencies"]) if task["dependencies"] else "None",
        "Can Parallelize With": ", ".join(task["parallel_with"]) if task["parallel_with"] else "None",
        "Automation": task["automation"],
        "Optional": "Yes" if task.get("optional") else "No"
    })

df_dependencies = pd.DataFrame(dependency_data)
print("\n📊 Task Dependency Table")
print("=" * 150)
print(df_dependencies.to_string(index=False))
print("=" * 150)

# Summary statistics
automated_count = sum(1 for t in workflow_tasks if t["automation"] == "automated" and not t.get("optional"))
manual_count = sum(1 for t in workflow_tasks if t["automation"] == "manual" and not t.get("optional"))
hybrid_count = sum(1 for t in workflow_tasks if t["automation"] == "hybrid" and not t.get("optional"))

print(f"\n📈 Task Statistics")
print(f"   Total Tasks: {len([t for t in workflow_tasks if not t.get('optional')])}")
print(f"   Automated: {automated_count} (fully automated)")
print(f"   Hybrid: {hybrid_count} (Playwright + manual fallback)")
print(f"   Manual: {manual_count} (user input required)")
print(f"   Parallelizable: 4 tasks in Phase 1")
print(f"   Sequential/Blocking: 6 tasks across Phases 2-6")


📊 Task Dependency Table
Task ID                      Name  Phase Duration (min)         Depends On Can Parallelize With Automation Optional
    1.1  Anthropic Account Signup      1            3-5               None        1.2, 1.3, 1.4     hybrid       No
    1.2     Vercel Account Signup      1            3-5               None        1.1, 1.3, 1.4     hybrid       No
    1.3     Convex Account Signup      1            3-5               None        1.1, 1.2, 1.4     hybrid       No
    1.4   GitHub Token Generation      1            2-3               None        1.1, 1.2, 1.3     hybrid       No
    2.1      Domain Configuration      2            2-3 1.1, 1.2, 1.3, 1.4                 None     manual       No
    2.2  Deployment Configuration      2            1-2                2.1                 None     manual       No
    3.1     Deploy Convex Backend      3            5-7           1.3, 2.2                 None  automated       No
    4.1     Create Vercel Project      4       

## Section 5: Export to Multiple Formats

Generate outputs suitable for different use cases:
1. **Markdown** - Human-readable question list for documentation
2. **JSON** - SurveyQuestion records for QuestionManager import
3. **SQL** - Direct database insertion

In [6]:
# 1. Generate Markdown output
markdown_output = "# DendWrite AI Setup Questions\n\n"
markdown_output += "Complete list of setup questions organized by topic for QuestionManager.\n\n"

for topic in survey_questions.keys():
    markdown_output += f"## {topic}\n\n"
    questions = survey_questions[topic]
    for idx, q_data in enumerate(questions, 1):
        markdown_output += f"{idx}. {q_data['q']}\n"
        markdown_output += f"   - Type: {q_data['type']}\n"
        if q_data.get('encrypted'):
            markdown_output += f"   - 🔒 Encrypted\n"
        if q_data.get('conditional'):
            markdown_output += f"   - Conditional: {q_data['conditional']}\n"
        markdown_output += "\n"

print("✓ Markdown output generated")
print(f"  Length: {len(markdown_output)} characters")

# 2. Generate JSON export for QuestionManager
json_export = {
    "export_metadata": {
        "created": datetime.now().isoformat(),
        "total_questions": len(survey_records),
        "topics": len(survey_questions),
        "format_version": "1.0"
    },
    "survey_questions": survey_records,
    "summary": {
        "topics": list(survey_questions.keys()),
        "encrypted_fields": [r for r in survey_records if "encrypted" in r.get("SqlDataType", "").lower()]
    }
}

print("\n✓ JSON export generated")
print(f"  Records: {len(json_export['survey_questions'])}")
print(f"  Encrypted fields: {len(json_export['summary']['encrypted_fields'])}")

# 3. Generate Task Dependency JSON
tasks_export = {
    "export_metadata": {
        "created": datetime.now().isoformat(),
        "total_tasks": len(workflow_tasks),
        "phases": 6,
        "critical_path_minutes": total_time
    },
    "tasks": workflow_tasks,
    "statistics": {
        "total_tasks": len(workflow_tasks),
        "automated": automated_count,
        "hybrid": hybrid_count,
        "manual": manual_count,
        "parallelizable": 4,
        "sequential": len([t for t in workflow_tasks if not t.get("optional")])
    }
}

print("\n✓ Task dependency JSON generated")
print(f"  Tasks: {len(tasks_export['tasks'])}")
print(f"  Critical path: {tasks_export['export_metadata']['critical_path_minutes']} minutes")

✓ Markdown output generated
  Length: 2108 characters

✓ JSON export generated
  Records: 30
  Encrypted fields: 5

✓ Task dependency JSON generated
  Tasks: 11
  Critical path: 35 minutes


In [8]:
# Save survey records to file (belt and suspenders approach)
import os

# Use specs folder (following project convention)
output_dir = "specs/005-survey-questions"
os.makedirs(output_dir, exist_ok=True)

# Save JSON export with timestamp
json_file = os.path.join(output_dir, f"survey_questions_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
with open(json_file, 'w') as f:
    json.dump(json_export, f, indent=2, default=str)

print(f"\n✓ Survey questions exported to: {json_file}")
print(f"  Total records: {len(survey_records)}")

# Also save markdown (UTF-8 for emoji support)
md_file = os.path.join(output_dir, f"survey_questions_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md")
with open(md_file, 'w', encoding='utf-8') as f:
    f.write(markdown_output)

print(f"✓ Markdown exported to: {md_file}")

# Save task definitions too
tasks_file = os.path.join(output_dir, f"workflow_tasks_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
# Note: workflow_tasks is defined in the next cell, so we'll save it there

print(f"\n📁 All survey artifacts saved to: {output_dir}/")



✓ Survey questions exported to: specs/005-survey-questions\survey_questions_20260116_114135.json
  Total records: 30
✓ Markdown exported to: specs/005-survey-questions\survey_questions_20260116_114135.md

📁 All survey artifacts saved to: specs/005-survey-questions/


## Section 6: Workflow Mermaid Diagram

Visual representation of the setup workflow showing:
- 6 phases (account creation → testing)
- Parallel opportunities in Phase 1
- Sequential dependencies in Phases 3-4
- Optional DNS configuration in Phase 5
- Final validation in Phase 6

```mermaid
graph TD
    START([User Initiates Setup]) --> PHASE1[Phase 1: Account Creation<br/>~5 min total<br/>4 tasks in parallel]
    
    PHASE1 --> ANTH["1.1 Anthropic<br/>Signup"]
    PHASE1 --> VERCEL["1.2 Vercel<br/>Signup"]
    PHASE1 --> CONVEX["1.3 Convex<br/>Signup"]
    PHASE1 --> GITHUB["1.4 GitHub<br/>Token"]
    
    ANTH --> ANTH_DONE["✓ API Key<br/>Ready"]
    VERCEL --> VERCEL_DONE["✓ Token<br/>Ready"]
    CONVEX --> CONVEX_DONE["✓ Deployment<br/>Key Ready"]
    GITHUB --> GITHUB_DONE["✓ PAT<br/>Ready"]
    
    ANTH_DONE --> PHASE2[Phase 2: Config<br/>~5 min]
    VERCEL_DONE --> PHASE2
    CONVEX_DONE --> PHASE2
    GITHUB_DONE --> PHASE2
    
    PHASE2 --> DOMAIN["2.1 Domain<br/>Config"]
    DOMAIN --> DEPLOY["2.2 Deployment<br/>Config"]
    
    DEPLOY --> PHASE3[Phase 3: Backend<br/>~5 min]
    PHASE3 --> CONVEX_DEPLOY["3.1 Deploy<br/>Convex"]
    
    CONVEX_DEPLOY --> CONVEX_URL["Get Production<br/>Convex URL"]
    CONVEX_URL --> PHASE4[Phase 4: Frontend<br/>~5 min]
    
    PHASE4 --> VERCEL_SETUP["4.1 Create<br/>Vercel Project"]
    VERCEL_SETUP --> VERCEL_DEPLOY["4.2 Deploy<br/>Frontend"]
    
    VERCEL_DEPLOY --> PHASE5[Phase 5: Domain<br/>Optional]
    PHASE5 --> DNS_CHECK{Custom<br/>Domain?}
    DNS_CHECK -->|Yes| DNS["5.1 Configure<br/>DNS"]
    DNS_CHECK -->|No| SKIP["Skip DNS"]
    
    DNS --> PHASE6[Phase 6: Testing<br/>~5 min]
    SKIP --> PHASE6
    
    PHASE6 --> TEST["6.1 End-to-End<br/>Testing"]
    TEST --> SUCCESS([✓ Setup Complete!<br/>27-32 minutes])
    
    style START fill:#90EE90
    style SUCCESS fill:#90EE90
    style PHASE1 fill:#87CEEB
    style PHASE2 fill:#87CEEB
    style PHASE3 fill:#87CEEB
    style PHASE4 fill:#87CEEB
    style PHASE5 fill:#FFB6C1
    style PHASE6 fill:#FFB6C1
```

**Key Benefits:**
- ✓ Phase 1: 4 parallel tasks → 5 min (vs 12 min sequential)
- ✓ Phase 3-4: Convex → Vercel (dependency chain, can't parallelize further)
- ✓ Phase 5: DNS async (doesn't block usage, can complete anytime)
- ✓ Critical path: 27-32 minutes including optional DNS setup

## Section 7: Summary & Next Steps

### What We've Created (Phase 0 Plan)

| Artifact | Format | Purpose |
|----------|--------|---------|
| **27 Setup Questions** | Topic-organized, Type-mapped | Questions to ask customers during deployment |
| **SurveyQuestion Records** | JSON | Import into QuestionManager database |
| **10 Workflow Tasks** | Dependency graph + timing | Blueprint for execution phases |
| **Mermaid Diagram** | Flow visualization | Understanding of critical path & dependencies |
| **Timeline Analysis** | Critical path calculation | Realistic expectations (27-32 min) |

### Phase 0 Validation Strategy

Our 27 questions are **first-guess assumptions** about what customers need to provide. As you execute the deployment:

1. **Document what works** - Mark which questions customers actually need to answer
2. **Document what fails** - Identify 2FA/CAPTCHA/rate-limit barriers
3. **Record actual timing** - Compare estimates vs. reality (5 min estimate might be 8 min with rate limits)
4. **Identify automation blockers** - Which steps must stay manual vs. could be automated
5. **Refine for next customer** - Build improved questionnaire and task list

### Phase 5 Execution (Next Steps)

When you're ready to execute:

1. **Run the 10 tasks** against your own DendWriteAI setup
2. **Answer the 27 questions** as if you're the first customer
3. **Document discoveries** in parallel:
   - ✅ What succeeded without manual intervention
   - ❌ What failed (2FA, CAPTCHA, rate limits?)
   - ⏱️ Actual duration (vs. estimated)
   - 🔧 Manual workarounds needed

4. **Record in specs/005/** - Implementation documentation for next iteration

### Key Metrics from This Plan

- **Total Questions**: 27 across 6 topics (Anthropic, Vercel, Convex, GitHub, Domain, Deployment)
- **Total Tasks**: 10 (6 essential, 1 optional DNS)
- **Parallelizable**: 4 account creation tasks in Phase 1
- **Critical Path**: 27-32 minutes (estimated, requires validation)
- **Automation**: 6 fully automated, 2 hybrid (Playwright + fallback), 2 manual

### Important Notes

- **This is a hypothesis**, not a guarantee - execution will teach us what's real
- **2FA is a blocker** - any vendor with mandatory 2FA will need manual intervention or special handling
- **Rate limits matter** - rapid API calls during automation may hit vendor rate limits
- **Domain DNS is async** - doesn't block MVP go-live (can happen anytime)
- **First customer = best feedback** - you're the ideal first test case

### Success Criteria When Done

✅ Answers to all 27 questions documented (what worked, what didn't)  
✅ 10 tasks executed against your setup (success/failure for each)  
✅ Actual timing recorded (total time, per-phase breakdown)  
✅ List of automation blockers identified  
✅ Plan for MVP deployment captured in specs/005/  


## Appendix: AI Question Type Inference (Claude Skills)

### Overview

This appendix demonstrates how to use Claude Skills to automatically infer question metadata (type, validation rules, help text) from raw question text provided by customers. Instead of manually creating 27 survey questions with all attributes, a customer provides a simple list, and the skill generates complete metadata.

### Why This Matters for QuestionManager

- **Eliminates data entry**: Customers don't specify types manually
- **Consistent quality**: Same inference rules applied to all questions
- **Feedback loop**: Results improve as the skill learns from corrections
- **Scalable**: Works for 10, 100, or 1000 questions
